In [4]:
import os
import glob
import json
from uuid import uuid4

from pydantic import BaseModel, Field, field_validator

import pymongo
from pymongo import MongoClient

import sqlite3

In [5]:
base_dir = './'
os.listdir(base_dir)

['features',
 'media-info',
 'download_file_from_gdrive.sh',
 'dev_write_data_to_db.ipynb',
 'objects',
 's_optimized_keyframes',
 'download_file_from_kaggle.sh',
 'vid_url.json',
 'keyframes.csv',
 'url_fps.json',
 'README.md']

In [6]:
os.listdir(os.path.join(base_dir, 'media-info'))

['L02_V008.json',
 'L15_V023.json',
 'L24_V017.json',
 'L19_V006.json',
 'L10_V017.json',
 'L05_V011.json',
 'L01_V017.json',
 'L20_V006.json',
 'L02_V025.json',
 'L15_V021.json',
 'L01_V008.json',
 'L17_V002.json',
 'L24_V045.json',
 'L09_V025.json',
 'L13_V019.json',
 'L12_V022.json',
 'L15_V011.json',
 'L24_V005.json',
 'L02_V003.json',
 'L21_V019.json',
 'L05_V021.json',
 'L12_V027.json',
 'L13_V025.json',
 'L24_V031.json',
 'L22_V017.json',
 'L04_V028.json',
 'L15_V022.json',
 'L14_V020.json',
 'L21_V026.json',
 'L11_V017.json',
 'L21_V022.json',
 'L04_V024.json',
 'L02_V007.json',
 'L16_V024.json',
 'L20_V009.json',
 'L06_V009.json',
 'L07_V004.json',
 'L07_V019.json',
 'L07_V002.json',
 'L24_V038.json',
 'L05_V008.json',
 'L13_V026.json',
 'L24_V013.json',
 'L06_V003.json',
 'L24_V039.json',
 'L13_V024.json',
 'L18_V024.json',
 'L17_V022.json',
 'L02_V018.json',
 'L18_V023.json',
 'L03_V025.json',
 'L08_V022.json',
 'L19_V017.json',
 'L02_V017.json',
 'L21_V006.json',
 'L12_V012

In [7]:
with open(os.path.join(base_dir, 'media-info', 'L02_V008.json'), 'r') as f: 
    json_obj = json.load(f)

json_obj

{'author': '60 Giây Official',
 'channel_id': 'UCRjzfa1E0gA50lvDQipbDMg',
 'channel_url': 'https://www.youtube.com/channel/UCRjzfa1E0gA50lvDQipbDMg',
 'description': '60 Giây Chiều - Ngày 24/10/2023 - HTV Tin Tức Mới Nhất\n► Đăng ký KÊNH để xem Tin Tức Mới Nhất: https://bit.ly/2HoUna4\nKênh Tin Tức Thời Sự 60 Giây Là Kênh Tổng Hợp Tin Tức - Sự Kiện - Giải trí Nhanh Nhất Việt Nam.\n------------\nXem TV online tại Truyền hình HTVC:\n✅ Web / Wap mobile :\n► https://hplus.com.vn\n► https://htvc.com.vn/\n► https://htvc.tv/\n✅ App mobile/TV HTVC :\n► IOS: https://goo.gl/Dih3DB\n► Android: https://goo.gl/XGBzxg\n► Smart TV /STB : Tải HTVC for Android TV\n.\n✅ Theo dõi HTVC trên Mạng xã hội :\n- Facebook Show : https://www.facebook.com/HPlusOfficial\n- Instagram Show : https://www.instagram.com/htvc_truyen_hinh_internet/\n- Lotus Show : https://lotus.vn/w/profile/18444255300272761.htm\n- Tiktok News : https://www.tiktok.com/@htvc.tintuc\n- Tiktok Phim : https://www.tiktok.com/@htvc.phimtruyen\

In [8]:
json_obj.keys()

dict_keys(['author', 'channel_id', 'channel_url', 'description', 'keywords', 'length', 'publish_date', 'thumbnail_url', 'title', 'watch_url'])

In [9]:
kw = ""
for word in json_obj['keywords']: 
    kw += word + " | "

kw

'HTV Tin tức | HTV News | chuong trinh 60 giay | chuong trinh 60s | ban tin 60 giay | ban tin 60s | 60 giay | 60s | 60 giay hom nay | Tin tức 60 giây | TIN TUC 60 GIAY | TIN TỨC 60 GI Y | tin tuc 60 giay | tin tức | 60 giây | Tin Tức | TIN TUC | 60 GIAY | Tin Tức Mới | Tin Tức Mới Nhất | 60 Giây | HTV Tin Tức | HTV news | Xem tin tức mới nhất | Xem tin tức | Hôm nay xem gì | 60 giây Sáng | 60 giây chiều | Bản tin 60s | Tin tức thời sự việt nam | Tin tuc thoi su Viet nam | '

# Schema

In [10]:
class VideoMetadata(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid4()))  # Generate and convert UUID to string
    title: str = Field(..., description="Title of the YouTube video")
    description: str = Field(..., description="Description of the video")
    publish_date: str = Field(..., description="Publication date of the video")
    thumbnail_url: str = Field(..., description="URL of the video thumbnail")
    watch_url: str = Field(..., description="URL to watch the video")
    channel_id: str
    channel_url: str
    author: str
    keywords: str
    length: int
    channel_title: str = Field(None, alias="title")  # For consistency with other fields

    @field_validator('publish_date', mode='before')
    def validate_publish_date(cls, v):
        # Add custom validation for publish_date format if needed
        # For example, using a date parser library
        return v

    @field_validator('keywords', mode='before')
    def validate_keywords(cls, v):
        kw = ""
        v = list(set(v))
        for word in v: 
            kw += word + " | "
        # return list(set(v))
        return kw

class VideoObjects(BaseModel): 
    video_name: str = Field(..., description="The name of the video")
    dtd_objects: list[str] = Field(..., description="A list of detected objects in the video")
    no_dtd_objects: list[int] = Field(..., description="The number of detected objects in the video")

In [11]:
vd_metadata = VideoMetadata(**json_obj)
vd_metadata.dict()

{'id': '679baeb9-a1ae-48f6-a86c-66725da79307',
 'title': '60 Giây Chiều - Ngày 24102023 - HTV Tin Tức Mới Nhất',
 'description': '60 Giây Chiều - Ngày 24/10/2023 - HTV Tin Tức Mới Nhất\n► Đăng ký KÊNH để xem Tin Tức Mới Nhất: https://bit.ly/2HoUna4\nKênh Tin Tức Thời Sự 60 Giây Là Kênh Tổng Hợp Tin Tức - Sự Kiện - Giải trí Nhanh Nhất Việt Nam.\n------------\nXem TV online tại Truyền hình HTVC:\n✅ Web / Wap mobile :\n► https://hplus.com.vn\n► https://htvc.com.vn/\n► https://htvc.tv/\n✅ App mobile/TV HTVC :\n► IOS: https://goo.gl/Dih3DB\n► Android: https://goo.gl/XGBzxg\n► Smart TV /STB : Tải HTVC for Android TV\n.\n✅ Theo dõi HTVC trên Mạng xã hội :\n- Facebook Show : https://www.facebook.com/HPlusOfficial\n- Instagram Show : https://www.instagram.com/htvc_truyen_hinh_internet/\n- Lotus Show : https://lotus.vn/w/profile/18444255300272761.htm\n- Tiktok News : https://www.tiktok.com/@htvc.tintuc\n- Tiktok Phim : https://www.tiktok.com/@htvc.phimtruyen\n- Tiktok Show : https://www.tiktok.c

In [12]:
def video_metadata_factory(cursor, row): 
    fields = [column[0] for column in cursor.description]
    fields = {key: value for key, value in zip(fields, row)}
    fields['keywords'] = fields['keywords'].strip().split(' | ')    # Parse string into list
    return fields

# Write data

In [13]:
con = sqlite3.connect("metadata.db")
con.execute("CREATE TABLE IF NOT EXISTS video_metadata (id TEXT PRIMARY KEY, title TEXT NOT NULL, description TEXT NOT NULL, publish_date TEXT NOT NULL, thumbnail_url TEXT NOT NULL, watch_url TEXT NOT NULL, channel_id TEXT NOT NULL, channel_url TEXT NOT NULL, author TEXT NOT NULL, keywords TEXT NOT NULL, length INTEGER NOT NULL, channel_title TEXT);")


In [14]:
metadata_dir = os.listdir(os.path.join(base_dir, 'media-info'))
data_objects = []
for metadata in metadata_dir: 
    with open(os.path.join(base_dir, 'media-info', metadata), 'r') as f: 
        json_obj = json.load(f)

    # Using this to compress the datasource into our desired configurations
    data = VideoMetadata(**json_obj)
    data_objects.append(data)

len(data_objects)
    

726

In [15]:
data_objects[10].dict()

{'id': 'fd14357e-a1e6-4849-a44b-bd609906aee9',
 'title': '60 Giây Sáng - Ngày 24102023 - HTV Tin Tức Mới Nhất',
 'description': '60 Giây Sáng - Ngày 24/10/2023 - HTV Tin Tức Mới Nhất\n► Đăng ký KÊNH để xem Tin Tức Mới Nhất: https://bit.ly/2HoUna4\nKênh Tin Tức Thời Sự 60 Giây Là Kênh Tổng Hợp Tin Tức - Sự Kiện - Giải trí Nhanh Nhất Việt Nam.\n------------\nXem TV online tại Truyền hình HTVC:\n✅ Web / Wap mobile :\n► https://hplus.com.vn\n► https://htvc.com.vn/\n► https://htvc.tv/\n✅ App mobile/TV HTVC :\n► IOS: https://goo.gl/Dih3DB\n► Android: https://goo.gl/XGBzxg\n► Smart TV /STB : Tải HTVC for Android TV\n.\n✅ Theo dõi HTVC trên Mạng xã hội :\n- Facebook Show : https://www.facebook.com/HPlusOfficial\n- Instagram Show : https://www.instagram.com/htvc_truyen_hinh_internet/\n- Lotus Show : https://lotus.vn/w/profile/18444255300272761.htm\n- Tiktok News : https://www.tiktok.com/@htvc.tintuc\n- Tiktok Phim : https://www.tiktok.com/@htvc.phimtruyen\n- Tiktok Show : https://www.tiktok.com

In [16]:
data_holder = []
for data_obj in data_objects: 
    data_holder.append(data_obj.dict())

data_holder = tuple(data_holder)
len(data_holder)

726

In [17]:
data_holder[1]

{'id': 'a0e64b8c-95cc-4e69-ad1e-3b746c78dac2',
 'title': '60 Giây Sáng - Ngày 09052024 - HTV Tin Tức Mới Nhất 2024',
 'description': '60 Giây Sáng - Ngày 09/05/2024 - HTV Tin Tức Mới Nhất 2024\n► Đăng ký KÊNH để xem Tin Tức Mới Nhất: https://bit.ly/2HoUna4\nKênh Tin Tức Thời Sự 60 Giây Là Kênh Tổng Hợp Tin Tức - Sự Kiện - Giải trí Nhanh Nhất Việt Nam.\n\n©️ Bản quyền Tin Tức thuộc HTV - Đài Truyền hình TP.HCM\n©️ The copyright of this video belongs to HTV - Ho Chi Minh City Television\n#TintucthoisuVietnam\n#HTVTintuc #HTVnews #HTV60s #60Giay #60s\n------------\nXem TV online tại Truyền hình HTVC:\n✅ Web / Wap mobile :\n► https://hplus.com.vn\n► https://htvc.com.vn/\n► https://htvc.tv/\n✅ App mobile/TV HTVC :\n► IOS: https://goo.gl/Dih3DB\n► Android: https://goo.gl/XGBzxg\n► Smart TV /STB : Tải HTVC for Android TV\n.\n✅ Theo dõi HTVC trên Mạng xã hội :\n- Facebook Show : https://www.facebook.com/HPlusOfficial\n- Instagram Show : https://www.instagram.com/htvc_truyen_hinh_internet/\n- L

In [18]:
con.execute("SELECT * FROM video_metadata;").fetchall()

[]

In [19]:
con.executemany("INSERT INTO video_metadata \
                VALUES (:id, :title, :description, :publish_date, :thumbnail_url, :watch_url, :channel_id, :channel_url, :author, :keywords, :length, :channel_title);",data_holder)

In [20]:
con.row_factory = video_metadata_factory

In [21]:
con.row_factory = sqlite3.Row

In [22]:
title = '60 Giây Chiều - Ngày 28032024 - HTV Tin Tức Mới Nhất 2024'
con.execute("SELECT * FROM video_metadata WHERE title = ?", (title, )).fetchmany()[0]